In [1]:
import shapefile

In [9]:
file = shapefile.Reader("data/SP_Setores_2021/SP_Setores_2021.shp")
new_file = shapefile.Writer("data/SP_Setores_2021/sp.shp")
new_file.fields = file.fields[1:]

In [12]:
for shape in file.shapeRecords():
    fields = dict(zip([field[0] for field in file.fields[1:]], shape.record))
    if fields["NM_MUN"] == "São Paulo":
        new_file.record(*shape.record)
        new_file.shape(shape.shape)

In [14]:
new_file.close()

In [15]:
file = shapefile.Reader("data/SP_Setores_2021/sp.shp")
print(file.numRecords)

55184
